In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv(r'train.csv' , encoding="latin-1")
data.head()

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26


In [3]:
data['text'] = data['text'].astype(str)

In [4]:
import re
import unicodedata
def clean_text(text):
    text = unicodedata.normalize("NFKD", text)
    text = text.encode("ascii", "ignore").decode("utf-8", "ignore")
    text = re.sub(r'[•*|►▪●]', ' ', text)
    text = re.sub(r'[\r\n\t]', ' ', text)
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
    text = text.lower()
    text = re.sub(r'\s+', ' ', text).strip()
    return text
data['text'] = data['text'].apply(clean_text)

In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
le = LabelEncoder()
data['sentiment'] = le.fit_transform(data['sentiment'])

In [6]:
x = data['text'].iloc[:1000].tolist()
y = data['sentiment'].iloc[:1000].astype(str).tolist()

In [7]:
print(x,y)

['i d have responded if i were going', 'sooo sad i will miss you here in san diego', 'my boss is bullying me', 'what interview leave me alone', 'sons of why couldn t they put them on the releases we already bought', 'http www dothebouncy com smf some shameless plugging for the best rangers forum on earth', '2am feedings for the baby are fun when he is all smiles and coos', 'soooo high', 'both of you', 'journey wow u just became cooler hehe is that possible', 'as much as i love to be hopeful i reckon the chances are minimal p i m never gonna get my cake and stuff', 'i really really like the song love story by taylor swift', 'my sharpie is running dangerously low on ink', 'i want to go to music tonight but i lost my voice', 'test test from the lg env2', 'uh oh i am sunburned', 's ok trying to plot alternatives as we speak sigh', 'i ve been sick for the past few days and thus my hair looks wierd if i didnt have a hat on it would look http tinyurl com mnf4kw', 'is back home now gonna miss 

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)
sequence = tokenizer.texts_to_sequences(x)

vocab_size = len(tokenizer.word_index)
print(vocab_size)

import numpy as np
max_len = max(len(seq) for seq in sequence)
print('max_len is : ' , max_len)

3067
max_len is :  31


In [9]:
max_len = 31
x = pad_sequences(sequence , maxlen=max_len)
print("Sequences:\n", x)



Sequences:
 [[   0    0    0 ...    1  126   42]
 [   0    0    0 ...   12  700 1082]
 [   0    0    0 ...   11 1083   15]
 ...
 [   0    0    0 ...  286 3063 1055]
 [   0    0    0 ...  285  548   23]
 [   0    0    0 ...    5   13 3067]]


In [10]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

# BEFORE train_test_split
le = LabelEncoder()
y_num = le.fit_transform(y)          # y_num is int labels 0..K-1

x_train, x_test, y_train, y_test = train_test_split(
    x, y_num, test_size=0.2, random_state=2
)

x_train = np.array(x_train, dtype="int32")  # tokens indices
x_test  = np.array(x_test,  dtype="int32")
y_train = np.array(y_train, dtype="int32")  # labels
y_test  = np.array(y_test,  dtype="int32")



In [14]:
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, SimpleRNN, LeakyReLU, PReLU, ELU, Activation, Embedding
from tensorflow.keras import optimizers, regularizers
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

# ===== DEFINE CUSTOM HARD SWISH FUNCTION =====
def hard_swish(x):
    """Hard Swish activation function"""
    return x * tf.nn.relu6(x + 3) / 6

# ===== PAD YOUR SEQUENCES FIRST! =====
vocab_size = 3067
TIMESTEPS = 20  # Truncate from 720

# THIS IS CRITICAL - PAD BEFORE EVERYTHING ELSE!
# x_train = pad_sequences(x_train, maxlen=TIMESTEPS, padding='post', truncating='post')
# x_test = pad_sequences(x_test, maxlen=TIMESTEPS, padding='post', truncating='post')

# print(f"After padding - x_train shape: {x_train.shape}")
# print(f"After padding - x_test shape: {x_test.shape}")


# ===== HYPERMODEL CLASS WITH EMBEDDING =====
class RNNHyperModel(kt.HyperModel):
    def __init__(self, timesteps, vocab_size):
        self.timesteps = timesteps
        self.vocab_size = vocab_size
    
    def build(self, hp):
        model = Sequential()

        # Tune batch_size and epochs
        hp.Int('batch_size', 16, 128, step=16, default=32)
        hp.Int('epochs', 10, 100, step=10, default=50)

        # ===== EMBEDDING LAYER - CRITICAL FOR TEXT DATA =====
        embedding_dim = hp.Int('embedding_dim', 50, 256, step=50)
        model.add(Embedding(
            input_dim=self.vocab_size + 1,  # +1 for padding token
            output_dim=embedding_dim,
            input_length=self.timesteps,
            mask_zero=True
        ))

        # Tune number of RNN layers
        num_layers = hp.Int("num_rnn_layers", 1, 5, step=1)
        
        for i in range(num_layers):
            units = hp.Int(f"rnn_units_{i}", 16, 256, step=16)
            
            activation = hp.Choice(
                f"rnn_activation_{i}",
                ["tanh", "relu", "sigmoid", "leaky_relu", "prelu", "elu",
                 "selu", "swish", "gelu", "mish", "hard_swish",
                 "hard_sigmoid", "linear"]
            )
            
            dropout = hp.Float(f"rnn_dropout_{i}", 0.0, 0.5, step=0.05)
            recurrent_dropout = hp.Float(f"rnn_recurrent_dropout_{i}", 0.0, 0.3, step=0.05)
            l2_reg = hp.Float(f"rnn_l2_{i}", 0.0, 0.01, step=0.001)
            
            return_sequences = (i < num_layers - 1)
            
            # For custom activations, use 'linear' in SimpleRNN
            if activation in ["leaky_relu", "prelu", "elu", "swish", "gelu", "mish", "hard_swish"]:
                rnn_activation = "linear"
            else:
                rnn_activation = activation
            
            model.add(SimpleRNN(
                units=units,
                activation=rnn_activation,
                dropout=dropout,
                recurrent_dropout=recurrent_dropout,
                return_sequences=return_sequences,
                kernel_regularizer=regularizers.l2(l2_reg) if l2_reg > 0 else None,
                recurrent_regularizer=regularizers.l2(l2_reg) if l2_reg > 0 else None
            ))
            
            # Add custom activation layer if needed
            if activation == "leaky_relu":
                model.add(LeakyReLU())
            elif activation == "prelu":
                model.add(PReLU())
            elif activation == "elu":
                model.add(ELU())
            elif activation == "swish":
                model.add(Activation(tf.nn.swish))
            elif activation == "gelu":
                model.add(Activation(tf.nn.gelu))
            elif activation == "mish":
                model.add(Activation(lambda x: x * tf.math.tanh(tf.math.softplus(x))))
            elif activation == "hard_swish":
                model.add(Activation(hard_swish))  # Use custom hard_swish function

        # Optional Dense layers after RNN
        use_dense = hp.Boolean("use_dense_layers")
        if use_dense:
            num_dense = hp.Int("num_dense_layers", 1, 3, step=1)
            for i in range(num_dense):
                dense_units = hp.Int(f"dense_units_{i}", 16, 256, step=16)
                dense_activation = hp.Choice(
                    f"dense_activation_{i}",
                    ["sigmoid", "tanh", "relu", "leaky_relu", "prelu", "elu",
                     "selu", "swish", "gelu", "hard_sigmoid", "linear"]
                )
                l2_reg_dense = hp.Float(f"dense_l2_{i}", 0.0, 0.01, step=0.001)

                model.add(Dense(
                    units=dense_units,
                    kernel_regularizer=regularizers.l2(l2_reg_dense) if l2_reg_dense > 0 else None
                ))

                # Apply activation
                if dense_activation == "leaky_relu":
                    model.add(LeakyReLU())
                elif dense_activation == "prelu":
                    model.add(PReLU())
                elif dense_activation == "elu":
                    model.add(ELU())
                elif dense_activation == "swish":
                    model.add(Activation(tf.nn.swish))
                elif dense_activation == "gelu":
                    model.add(Activation(tf.nn.gelu))
                else:
                    model.add(Activation(dense_activation))

                dropout_rate = hp.Float(f"dense_dropout_{i}", 0.0, 0.5, step=0.05)
                if dropout_rate > 0:
                    model.add(Dropout(rate=dropout_rate))

        # Output layer
        model.add(Dense(1, activation="sigmoid"))

        # Tune optimizer
        optimizer_name = hp.Choice(
            "optimizer",
            ["sgd", "momentum", "nesterov", "adagrad", "adadelta",
             "rmsprop", "adam", "adamax", "nadam", "adamw"]
        )
        lr = hp.Float("learning_rate", 1e-5, 1e-2, sampling="log")

        if optimizer_name == "sgd":
            optimizer = optimizers.SGD(learning_rate=lr)
        elif optimizer_name == "momentum":
            optimizer = optimizers.SGD(learning_rate=lr, momentum=0.9)
        elif optimizer_name == "nesterov":
            optimizer = optimizers.SGD(learning_rate=lr, momentum=0.9, nesterov=True)
        elif optimizer_name == "adagrad":
            optimizer = optimizers.Adagrad(learning_rate=lr)
        elif optimizer_name == "adadelta":
            optimizer = optimizers.Adadelta(learning_rate=lr)
        elif optimizer_name == "rmsprop":
            optimizer = optimizers.RMSprop(learning_rate=lr)
        elif optimizer_name == "adam":
            optimizer = optimizers.Adam(learning_rate=lr)
        elif optimizer_name == "adamax":
            optimizer = optimizers.Adamax(learning_rate=lr)
        elif optimizer_name == "nadam":
            optimizer = optimizers.Nadam(learning_rate=lr)
        elif optimizer_name == "adamw":
            optimizer = optimizers.AdamW(learning_rate=lr)

        model.compile(
            optimizer=optimizer,
            loss="binary_crossentropy",
            metrics=["accuracy"]
        )
        return model


# ===== CUSTOM TUNER CLASS =====
class MyRNNTuner(kt.RandomSearch):
    def run_trial(self, trial, *args, **kwargs):
        kwargs['batch_size'] = trial.hyperparameters.get('batch_size')
        kwargs['epochs'] = trial.hyperparameters.get('epochs')
        return super(MyRNNTuner, self).run_trial(trial, *args, **kwargs)


# ===== CREATE HYPERMODEL INSTANCE =====
hypermodel = RNNHyperModel(timesteps=TIMESTEPS, vocab_size=vocab_size)

# ===== INITIALIZE TUNER =====
tuner = MyRNNTuner(
    hypermodel=hypermodel,
    objective="val_accuracy",
    max_trials=25,
    directory="rnn_tuner",
    project_name="full_rnn_tuning",
    overwrite=True
)

# ===== RUN HYPERPARAMETER SEARCH =====
print("Starting RNN hyperparameter search...")
print(f"Vocabulary size: {vocab_size}")
print(f"Sequence length (TIMESTEPS): {TIMESTEPS}")
print(f"x_train shape: {x_train.shape}")  # Should be (samples, 20)
print(f"Training samples: {len(x_train)}")
print(f"Test samples: {len(x_test)}")
print("-" * 60)

tuner.search(
    x_train,
    y_train,
    validation_data=(x_test, y_test),
    verbose=1
)

# ===== PRINT BEST HYPERPARAMETERS =====
best_hp = tuner.get_best_hyperparameters(1)[0]

print("\n" + "="*60)
print("BEST HYPERPARAMETERS")
print("="*60)

print("\n📝 Embedding Configuration:")
print(f"  Embedding Dimension: {best_hp.get('embedding_dim')}")

print("\n📊 Optimizer Configuration:")
print(f"  Optimizer: {best_hp.get('optimizer')}")
print(f"  Learning Rate: {best_hp.get('learning_rate'):.6f}")

print("\n🔄 RNN Architecture:")
print(f"  Number of RNN Layers: {best_hp.get('num_rnn_layers')}")

for i in range(best_hp.get("num_rnn_layers")):
    print(f"\n  RNN Layer {i+1}:")
    print(f"    Units: {best_hp.get(f'rnn_units_{i}')}")
    print(f"    Activation: {best_hp.get(f'rnn_activation_{i}')}")
    print(f"    Dropout: {best_hp.get(f'rnn_dropout_{i}'):.3f}")
    print(f"    Recurrent Dropout: {best_hp.get(f'rnn_recurrent_dropout_{i}'):.3f}")
    print(f"    L2 Regularization: {best_hp.get(f'rnn_l2_{i}'):.4f}")

print("\n🔗 Dense Layers:")
if best_hp.get("use_dense_layers"):
    print(f"  Number of Dense Layers: {best_hp.get('num_dense_layers')}")
    for i in range(best_hp.get("num_dense_layers")):
        print(f"\n  Dense Layer {i+1}:")
        print(f"    Units: {best_hp.get(f'dense_units_{i}')}")
        print(f"    Activation: {best_hp.get(f'dense_activation_{i}')}")
        print(f"    Dropout: {best_hp.get(f'dense_dropout_{i}'):.3f}")
        print(f"    L2 Regularization: {best_hp.get(f'dense_l2_{i}'):.4f}")
else:
    print("  No additional dense layers used")

print("\n⚙️ Training Configuration:")
print(f"  Batch Size: {best_hp.get('batch_size')}")
print(f"  Epochs: {best_hp.get('epochs')}")

# ===== BUILD AND EVALUATE BEST MODEL (FIXED) =====
print("\n" + "="*60)
print("BUILDING BEST MODEL FROM HYPERPARAMETERS")
print("="*60)

# Build model with best hyperparameters instead of loading
best_model = hypermodel.build(best_hp)
best_model.summary()

print("\n" + "="*60)
print("TRAINING BEST MODEL")
print("="*60)

# Train the best model
history = best_model.fit(
    x_train, 
    y_train,
    batch_size=best_hp.get('batch_size'),
    epochs=best_hp.get('epochs'),
    validation_data=(x_test, y_test),
    verbose=1
)

print("\n" + "="*60)
print("BEST MODEL EVALUATION")
print("="*60)
test_loss, test_accuracy = best_model.evaluate(x_test, y_test, verbose=0)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Optional: Save the final model
best_model.save('best_rnn_model.h5')
print("\n✅ Best model saved as 'best_rnn_model.h5'")


Trial 25 Complete [00h 01m 26s]
val_accuracy: 0.39500001072883606

Best val_accuracy So Far: 0.46000000834465027
Total elapsed time: 00h 28m 16s

BEST HYPERPARAMETERS

📝 Embedding Configuration:
  Embedding Dimension: 250

📊 Optimizer Configuration:
  Optimizer: adam
  Learning Rate: 0.002016

🔄 RNN Architecture:
  Number of RNN Layers: 2

  RNN Layer 1:
    Units: 16
    Activation: hard_swish
    Dropout: 0.150
    Recurrent Dropout: 0.250
    L2 Regularization: 0.0050

  RNN Layer 2:
    Units: 16
    Activation: tanh
    Dropout: 0.000
    Recurrent Dropout: 0.000
    L2 Regularization: 0.0000

🔗 Dense Layers:
  Number of Dense Layers: 1

  Dense Layer 1:
    Units: 208
    Activation: elu
    Dropout: 0.400
    L2 Regularization: 0.0090

⚙️ Training Configuration:
  Batch Size: 96
  Epochs: 100

BEST MODEL SUMMARY


C:\Users\ma516\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 20 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


ValueError: A total of 5 objects could not be loaded. Example error message for object <Embedding name=embedding, built=True>:

The shape of the target variable and the shape of the target value in `variable.assign(value)` must match. variable.shape=(3068, 250), Received: value.shape=(752, 50). Target variable: <Variable path=sequential/embedding/embeddings, shape=(3068, 250), dtype=float32, value=[[ 0.00616908  0.00518628 -0.02142142 ... -0.02530164  0.04671911
   0.03471209]
 [ 0.02611747  0.02662517 -0.04178997 ...  0.03355292 -0.01071769
  -0.04199447]
 [ 0.04809647 -0.02552225  0.01808413 ...  0.0447014   0.02023115
  -0.00804418]
 ...
 [-0.01382958  0.04034431 -0.00896523 ... -0.04311507 -0.02436051
  -0.03346868]
 [ 0.02773507 -0.01331367  0.04298138 ...  0.02857474 -0.04886701
   0.01316207]
 [-0.00663187  0.03691033  0.03003288 ... -0.03379162 -0.04845239
  -0.02582425]]>

List of objects that could not be loaded:
[<Embedding name=embedding, built=True>, <SimpleRNNCell name=si

In [26]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

line = "I`d have responded, if I were going"
seq = tokenizer.texts_to_sequences([line])

TIMESTEPS = 20
seq_padded = pad_sequences(seq, maxlen=TIMESTEPS, padding='post', truncating='post')

# Predict
y_pred_prob = best_model.predict(seq_padded)
y_pred_class = int(y_pred_prob[0][0] >= 0.5)

print(f"Predicted probability: {y_pred_prob[0][0]:.4f}")
print(f"Predicted class: {y_pred_class}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
Predicted probability: 1.0000
Predicted class: 1
